In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import pandas as pd

In [2]:
# sql_textに文字列として保存されたSQLを送信し、結果をDataFrameで返す
def get_dataframe(sql_text):
    try:
        chrome_service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=chrome_service)
        driver.implicitly_wait(10)

        page_url = 'https://erogamescape.dyndns.org/~ap2/ero/toukei_kaiseki/sql_for_erogamer_form.php'
        driver.get(page_url)
        sleep(5)
        # some process

        # SQL sending
        elem = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[3]/div/div[2]/form/div[1]/textarea')
        elem.clear()
        elem.send_keys(sql_text)
        elem.submit()
        sleep(10)

        # read html table
        elem_table = driver.find_element(By.XPATH, '//table')
        html = elem_table.get_attribute('outerHTML')
        dfs = pd.read_html(html)

        driver.close()
        return dfs[0].sort_values('id').reset_index()

    except Exception as e:
        print(e)
        driver.close()
        print("Fail")
        return pd.DataFrame()
    
# 再起的にディレクトリを作成
def mkdir(path):
    os.makedirs(path, exist_ok=True)

# DataFrameを保存
def save_dataframe(df, dir, filename):
    if not dir[-1]=='/':
        dir = dir+'/'
    mkdir(dir)
    df.to_csv(dir+filename)

In [ ]:
# # ブランド情報
# sql_text = 'SELECT * \
#              FROM brandlist'
# dir = './data/'
# filename='brandlist.csv'
# df = get_dataframe(sql_text)
# save_dataframe(df,dir,filename)


In [ ]:
# # ゲーム情報(全部・危険)
# # 重すぎるので5000個ずつ取ってくる
# sql_text = 'SELECT * \
#              FROM gamelist \
#                 WHERE gamelist.id>={} \
#                     AND gamelist.id<{}'
# dir = './data/'
# filename='gamelist.csv'
# number_from = 0
# d_number = 5000
# number_to = number_from+d_number
# df = get_dataframe(sql_text.format(number_from, number_to))
# number_from += d_number
# number_to += d_number
# while(True):
#     try:
#         next_df = get_dataframe(sql_text.format(number_from, number_to))
#         if(len(next_df))==0:
#             print("ended")
#             break
#         df = pd.concat([df, next_df])
#         number_from += d_number
#         number_to += d_number
#     except Exception as e:
#         print(e)
#         break

# df_sorted = df.sort_values('id').reset_index()
# save_dataframe(df_sorted,dir,filename)

In [ ]:
# # 非エロゲかつ中央値が存在する(=得点つけた人が１人以上)
# sql_text = 'SELECT * \
#              FROM gamelist \
#                 WHERE gamelist.erogame=false \
#                 AND gamelist.median IS NOT NULL'
# dir = './data/'
# filename='gamelist_safe.csv'
# df = get_dataframe(sql_text)
# save_dataframe(df,dir,filename)


In [ ]:
# # 全てのPOVのリスト。危険。
# sql_text = 'SELECT * \
#             FROM povlist \
#             ORDER BY povlist.id'
# dir = './data/'
# filename = 'povlist.csv'
# df = get_dataframe(sql_text)
# save_dataframe(df,dir,filename)

In [7]:
# 使えるかもしれないPOVのリスト。やや危険か。
using_pov_id_list = [2,3,6,7,9,10,13,16,23,27,28,29,30,33,34,35,36,38,40,42,43,44,45,46,47,48,50,51,52,54,55,57,58,61,\
    63,64,65,66,67,70,78,81,83,86,88,89,96,100,109,112,113,114,116,117,120,121,126,128,131,132,134,135,139,143,144,145,146,150,\
    151,156,157,163,167,169,172,174,176,177,182,192,194,198,214,219,229,235,240,244,245,253,254,258,261,262,263,270,276,287,310, \
    311,314,317,318,346,356,359,362,363,367,369,377,379,399,402,412,415,416,420,424,426,459,465,468,476,486,493,503,504,505,506, \
    507,508,510,511,512,513,519,520,523,524,526,527,530,532,533]

# メモ：10,16,48,67,88,89,214,276,510
using_pov_id_list = [10,48,67,88,89,214,240,276,415]
df = pd.read_csv('data/povlist.csv')
for i in df.index.values:
    if df.loc[i,'id'] in using_pov_id_list:
        print(df.loc[i,"id"])
        print(df.loc[i,'title'])

10
コメディ要素が強い笑えるゲーム
48
伏線がしっかり回収されているゲーム
67
感動した（泣いてしまった）ゲーム
88
鬱ゲー
89
（熱く燃える）熱血系ゲーム
214
魅力に溢れた幼なじみ
240
ツインテールが魅力
276
純愛物語
415
無表情な女性キャラが登場するゲーム


In [ ]:
# ゲームと結びつけるPOVを決定。
# using_pov_id_list_safe = [3,10,23,34,57,67,88,89,493]
using_pov_id_list_safe = [10,48,67,88,89,214,240,276,415]
# それぞれについてPOVが付けられたゲームのリストを入手
for i in using_pov_id_list_safe:
    print(i)
    print()
    sql_text = 'SELECT * \
                FROM povgroups p \
                INNER JOIN gamelist g \
                ON p.game=g.id \
                WHERE p.pov={} \
                AND g.erogame=false \
                AND g.median IS NOT NULL'.format(i)
    dir = './data/pov_to_game/'
    filename = 'pov{}_to_game.csv'.format(i)
    if filename in os.listdir('./data/pov_to_game/'):
        continue
    df = get_dataframe(sql_text)
    save_dataframe(df,dir,filename)

In [ ]:
file_list = os.listdir("./data/pov_to_game/")
for i in file_list:
    df = pd.read_csv("./data/pov_to_game/"+i,converters={'dmm_subsc':str})
    print(len(df))
    print(i)

In [12]:
# ゲームと結びつけたPOVのみによって構成されるCSVを作成
df_all_pov = pd.read_csv('./data/povlist.csv')
df = [(df_all_pov.loc[i,:]) for i in df_all_pov.index.values if df_all_pov.loc[i,'id']  in using_pov_id_list_safe]
dir='./data/'
filename = 'povlist_safe.csv'
df = pd.DataFrame(df)
save_dataframe(df,dir,filename)

In [13]:
# POVについての情報をgamelist_safe.csvに保存する。

df_gamelist = pd.read_csv("./data/gamelist_safe.csv")   # ゲームのリスト
df_pov_list = pd.read_csv("./data/povlist_safe.csv")    # povの種類のリスト   この二つの対応を取りたい

# どのPOVについてかのループ
for i in df_pov_list.index.values:
    pov_id = df_pov_list.loc[i,'id']
    print("pov"+str(pov_id))
    pov_file_path = './data/pov_to_game/pov{}_to_game.csv'.format(pov_id)
    df_pov_to_game = pd.read_csv(pov_file_path,converters={'dmm_subsc':str}) # 57のところでここがDtypeWarning出るからconvertersを指定
    
    pov_title = df_pov_list.loc[i,'system_title']      # 今見ているPOVのタイトル(system_titleでないとjavascriptで参照しづらい)
    point_list = []
    # どのゲームについてかのループ
    for j in df_gamelist.index.values:
        if j%1000==0:
            print(j)
        game_id = df_gamelist.loc[j,'id']
        df_pov_this_game_A = df_pov_to_game[(df_pov_to_game['game']==game_id) & (df_pov_to_game['rank']=='A')]
        df_pov_this_game_B = df_pov_to_game[(df_pov_to_game['game']==game_id) & (df_pov_to_game['rank']=='B')]
        df_pov_this_game_C = df_pov_to_game[(df_pov_to_game['game']==game_id) & (df_pov_to_game['rank']=='C')]
        count_A = len(df_pov_this_game_A)
        count_B = len(df_pov_this_game_B)
        count_C = len(df_pov_this_game_C)
        point = count_A*3+count_B*2+count_C
        point_list.append(point)
    df_gamelist[pov_title] = point_list


dir = './data/'
filename = 'gamelist_safe_added_pov.csv'     
save_dataframe(df_gamelist,dir,filename)

    

pov10
0
1000
2000
3000
4000
5000
6000
pov48
0
1000
2000
3000
4000
5000
6000
pov67
0
1000
2000
3000
4000
5000
6000
pov88
0
1000
2000
3000
4000
5000
6000
pov89
0
1000
2000
3000
4000
5000
6000
pov214
0
1000
2000
3000
4000
5000
6000
pov240
0
1000
2000
3000
4000
5000
6000
pov276
0
1000
2000
3000
4000
5000
6000
pov415
0
1000
2000
3000
4000
5000
6000


In [6]:
# # ゲームハードのリストの保存
# df = pd.read_csv("./data/gamelist_safe_added_pov.csv")
# s = set()
# for i in df.index.values:
#     model = df.loc[i,"model"]
#     s.add(model)
# print(s)
# df = pd.DataFrame()
# df["index"] = [i for i in range(0,len(s))]
# df["model"] = list(s)
# filename = 'model_list.csv'
# dir = './data/'
# save_dataframe(df,dir,filename)

{'PC', '3DS', 'XB360', 'XBO', 'Wii U', 'NS', 'WS', 'DC', 'PS5', 'Wii', 'NGP', 'PS4', 'PSP', 'PCE', 'iPhone', 'Android', 'XB', 'SFC', 'PSV', 'PS2', 'BROWSER', 'MCD', 'FC', 'MOBILE', 'PS', 'PS3', 'SS', 'GBA(GB)', 'NDS', 'iOS'}
